# Стандартная сессия
- с поддержкой MinIO и Postgres и с конфигом из `spark-defaults`
- Все параметры прописаны в `"\etl_project_v1\spark\conf\spark-defaults.conf"`

In [ ]:
from pyspark.sql import SparkSession

drivers = [
    "/opt/spark/external-jars/hadoop-aws-3.3.4.jar",
    "/opt/spark/external-jars/aws-java-sdk-bundle-1.12.262.jar",
    "/opt/spark/external-jars/wildfly-openssl-1.0.7.Final.jar",
    "/opt/spark/external-jars/postgresql-42.6.0.jar",
]

spark = (SparkSession.builder
         .appName("jupyter-spark")
         .master("spark://spark-master:7077")
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

---

### Дополнительные параметры конфигурации

In [ ]:
# Дополнительные оптимизации для "Full Power"
spark = (SparkSession.builder    
         .appName("Optimized for i5-14600K").master("spark://spark-master:7077")
         .config("spark.sql.adaptive.enabled", "true")          # Адаптивное выполнение запросов
         .config("spark.sql.adaptive.coalescePartitions.enabled", "true") # Объединение партиций
         .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") # Быстрая сериализация
         .config("spark.sql.shuffle.partitions", "200")         # Партиции для шаффлинга
         .config("spark.default.parallelism", "40")             # Параллелизм по умолчанию

# Конфигурация Spark для вашего железа:
spark = (SparkSession.builder
         .appName("Optimized for i5-14600K").master("spark://spark-master:7077")
         .config("spark.executor.instances", "2")  
         .config("spark.executor.cores", "8")      # Оставляем 2 ядра на систему
         .config("spark.executor.memory", "14g")   # Оставляем 4GB на систему
         .config("spark.driver.memory", "8g")      # Увеличиваем для больших collect()
         .config("spark.sql.adaptive.enabled", "true")
         .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
         .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "128MB")
         .config("spark.sql.shuffle.partitions", "100")  # 8 cores × 2 workers × 6 = 96
         .config("spark.default.parallelism", "96")
         .config("spark.memory.fraction", "0.8")
         .config("spark.memory.storageFraction", "0.3")
         .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
         .getOrCreate())

### Вывести текущую конфигурацию

In [ ]:
import pandas as pd

conf      = spark.sparkContext.getConf()
config_df = pd.DataFrame(conf.getAll(), columns=['Key', 'Value'])
config_df = config_df.sort_values('Key').reset_index(drop=True)
config_df

---

# Spark сессия с максимальной производительностью

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("Full Power")
         .master("spark://spark-master:7077")
         .config("spark.executor.instances", "2")
         .config("spark.executor.cores", "10")
         .config("spark.executor.memory", "16g")
         .config("spark.driver.memory", "4g")
         .config("spark.cores.max ", "20")
         .getOrCreate()
        )

In [ ]:
spark.stop()

---

# Коннектор к `MinIO`

### Использовать конфиг `MinIO` из `"\etl_project_v1\spark\conf\spark-defaults.conf"`

In [ ]:
from pyspark.sql import SparkSession

drivers = [
    "/opt/spark/external-jars/hadoop-aws-3.3.4.jar",
    "/opt/spark/external-jars/aws-java-sdk-bundle-1.12.262.jar",
    "/opt/spark/external-jars/wildfly-openssl-1.0.7.Final.jar",
]

spark = (SparkSession.builder
         .appName("MinIO Test")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

### Скачать дравера из интернета

In [ ]:
spark = (SparkSession.builder
         .appName("MinIO Test")
         .master("spark://spark-master:7077") 
         .config("spark.jars.packages", 
                 "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262")     
         .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
         .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.path.style.access", "true") 
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
         .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") 
         .getOrCreate()
        )

### Использовать скачанное из `"/opt/spark/external-jars/"`

In [ ]:
from pyspark.sql import SparkSession

drivers = [
    "/opt/spark/external-jars/hadoop-aws-3.3.4.jar",
    "/opt/spark/external-jars/aws-java-sdk-bundle-1.12.262.jar",
    "/opt/spark/external-jars/wildfly-openssl-1.0.7.Final.jar",
]

spark = (SparkSession.builder
         .appName("MinIO Test")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
         .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.path.style.access", "true") 
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
         .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") 
         .getOrCreate()
        )

### Проверить запись

In [ ]:
df = spark.range(10)
df.write.mode("overwrite").csv("s3a://learn-bucket/test_22")
print("✅ Успешно!")

## Файловая сессия с MinIO

In [ ]:
import boto3

s3_client = boto3.client('s3',
    endpoint_url='http://minio:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    region_name='us-east-1'
)

response = s3_client.list_buckets()
print(f"Доступные бакеты: {[bucket['Name'] for bucket in response['Buckets']]}")

In [ ]:
spark.stop()

---

# Коннектор к `PostgreSQL`

### Скачать из интернета

In [ ]:
# Чтение из PostgreSQL
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("PostgreSQL")
         .master("spark://spark-master:7077") 
         .config("spark.jars.packages", "org.postgresql:postgresql:42.5.0")
         .getOrCreate())

### Взять с локального диска

In [ ]:
# Чтение из PostgreSQL
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("PostgreSQL")
         .master("spark://spark-master:7077") 
         .config("spark.jars", "/opt/spark/external-jars/postgresql-42.6.0.jar")
         .getOrCreate())

# Качаем драйвера один раз

In [ ]:
import os
import requests

def download_spark_jars():
    """Скачивает необходимые JAR файлы в папку spark-jars"""
    
    jars_dir = "/opt/spark/external-jars/"
    os.makedirs(jars_dir, exist_ok=True)
    
    jars = {
        "hadoop-aws-3.3.4.jar": "https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar",
        "aws-java-sdk-bundle-1.12.262.jar": "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar",
        "wildfly-openssl-1.0.7.Final.jar": "https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.0.7.Final/wildfly-openssl-1.0.7.Final.jar",
        "postgresql-42.6.0.jar": "https://jdbc.postgresql.org/download/postgresql-42.6.0.jar"
    }
    
    for filename, url in jars.items():
        filepath = os.path.join(jars_dir, filename)
        if not os.path.exists(filepath):
            print(f"📥 Скачиваем {filename}...")
            response = requests.get(url)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            print(f"✅ {filename} загружен")
        else:
            print(f"✅ {filename} уже существует")

# Запустите эту функцию один раз
download_spark_jars()